# Markowitz Efficient Frontier

## 1. Imports

In [2]:
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.graph_objects as go
from tqdm import tqdm

from fetchData import fetch_raw_data_yf, get_matrices, getNasdaqStocks
from MonteCarloRBA import MonteCarloRBA


## 2. Fetch Data

### Get all Nasdaq Stocks

In [3]:
assets= [
    "AAPL",  # Apple Inc.
    "MSFT",  # Microsoft Corporation
    "AMZN",  # Amazon.com Inc.
    "GOOGL", # Alphabet Inc. (Google) Class A
    "GOOG",  # Alphabet Inc. (Google) Class C
    "META",    # Meta Platforms Inc (formerly Facebook)
    "TSLA",  # Tesla Inc
    "UA", # Berkshire Hathaway Inc. Class B
    "JPM",   # JPMorgan Chase & Co.
    "V",     # Visa Inc.
    "JNJ",   # Johnson & Johnson
    "WMT",   # Walmart Inc.
    "PG",    # Procter & Gamble Co.
    "UNH",   # UnitedHealth Group Inc.
    "MA",    # Mastercard Inc.
    "NVDA",  # NVIDIA Corporation
    "HD",    # Home Depot Inc.
    "BAC",   # Bank of America Corp
    "DIS",   # Walt Disney Co
    "PYPL",  # PayPal Holdings
    "VZ",    # Verizon Communications Inc.
    "ADBE",  # Adobe Inc.
    "CMCSA", # Comcast Corporation
    "NFLX",  # Netflix Inc.
    "KO",    # Coca-Cola Co
    "NKE",   # NIKE Inc.
    "PFE",   # Pfizer Inc.
    "MRK",   # Merck & Co., Inc.
    "PEP",   # PepsiCo, Inc.
    "T",     # AT&T Inc.
    "ABT",   # Abbott Laboratories
    "CRM",   # Salesforce.com Inc.
    "ORCL",  # Oracle Corporation
    "ABBV",  # AbbVie Inc.
    "CSCO",  # Cisco Systems, Inc.
    "INTC",  # Intel Corporation
    "TMO",   # Thermo Fisher Scientific Inc.
    "XOM",   # Exxon Mobil Corporation
    "ACN",   # Accenture plc
    "LLY",   # Eli Lilly and Company
    "COST",  # Costco Wholesale Corporation
    "MCD",   # McDonald's Corp
    "DHR",   # Danaher Corporation
    "MDT",   # Medtronic plc
    "NEE",   # NextEra Energy, Inc.
    "BMY",   # Bristol-Myers Squibb Company
    "QCOM",  # Qualcomm Inc
    "CVX",   # Chevron Corporation
    "WFC",   # Wells Fargo & Co
    "LMT",    # Lockheed Martin Corporation
    "GS",   # Goldman Sachs Group, Inc.
    "MS",   # Morgan Stanley
    "IBM",  # International Business Machines Corporation
    "GE",   # General Electric Company
    "F",    # Ford Motor Company
    "GM",   # General Motors Company
    "UBER", # Uber Technologies, Inc.
    "LYFT", # Lyft, Inc.
    "SNAP", # Snap Inc.
    "TWTR", # Twitter, Inc.
    "SPOT", # Spotify Technology S.A.
    "AMD",  # Advanced Micro Devices, Inc.
    "TXN",  # Texas Instruments Incorporated
    "BABA", # Alibaba Group Holding Limited
    "SAP",  # SAP SE
    "HON",  # Honeywell International Inc.
    "BA",   # Boeing Company
    "RTX",  # Raytheon Technologies Corporation
    "CAT",  # Caterpillar Inc.
    "DE",   # Deere & Company
    "MMM",  # 3M Company
    "DUK",  # Duke Energy Corporation
    "SO",   # Southern Company
    "EXC",  # Exelon Corporation
    "NEE",  # NextEra Energy, Inc.
    "AEP",  # American Electric Power Company, Inc.
    "SRE",  # Sempra Energy
    "ETN",  # Eaton Corporation plc
    "EMR",  # Emerson Electric Co.
    "SYY",  # Sysco Corporation
    "KR",   # Kroger Co.
    "GIS",  # General Mills, Inc.
    "K",    # Kellogg Company
    "CPB",  # Campbell Soup Company
    "MO",   # Altria Group, Inc.
    "PM",   # Philip Morris International Inc.
    "BTI",  # British American Tobacco plc
    "RDY",  # Dr. Reddy's Laboratories Ltd.
    "GILD", # Gilead Sciences, Inc.
    "BIIB", # Biogen Inc.
    "CELG", # Celgene Corporation
    "AMGN", # Amgen Inc.
    "SYK",  # Stryker Corporation
    "BSX",  # Boston Scientific Corporation
    "ISRG", # Intuitive Surgical, Inc.
    "ZBH",  # Zimmer Biomet Holdings, Inc.
    "EW",   # Edwards Lifesciences Corporation
    "RMD",  # ResMed Inc.
    "VRTX", # Vertex Pharmaceuticals Incorporated
    "REGN",  # Regeneron Pharmaceuticals, Inc.
]

len(assets)

100

In [4]:
raw_data, asset_errors, max_combination= fetch_raw_data_yf(assets)

raw_data

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  99 of 99 completed

5 Failed downloads:
['SPOT', 'LYFT', 'UBER']: YFPricesMissingError('possibly delisted; no price data found  (1d 2015-01-01 -> 2018-01-01) (Yahoo error = "Data doesn\'t exist for startDate = 1420088400, endDate = 1514782800")')
['TWTR', 'CELG']: YFTzMissingError('possibly delisted; no timezone found')


Omitted assets ( 5 ):  ['SPOT', 'TWTR', 'CELG', 'UBER', 'LYFT']
Time to fetch data: 3.69 seconds


Ticker,GILD_Close,V_Close,HD_Close,PEP_Close,ISRG_Close,COST_Close,DHR_Close,GOOG_Close,NFLX_Close,DUK_Close,...,UNH_Close,GS_Close,PFE_Close,MO_Close,GM_Close,BTI_Close,CPB_Close,BSX_Close,WFC_Close,MMM_Close
Date,,,,,,,,,,,,,,,,,,,,,
2017-03-02,52.150883,83.776299,122.044922,87.802948,82.507774,152.512024,73.970665,41.382812,139.529999,59.129272,...,148.590378,212.579742,23.527464,43.151474,32.196793,35.676353,46.359253,25.000000,46.771107,119.628609
2017-03-03,52.306160,84.022324,121.929489,88.161774,82.754440,145.896759,74.150032,41.305584,139.139999,58.806202,...,149.380753,214.129272,23.534281,43.065990,32.597549,35.391167,46.134552,24.980000,46.914494,119.263229
2017-03-06,51.847721,84.164291,121.368546,87.420181,81.744446,142.871872,73.518105,41.240818,141.940002,58.777477,...,149.824814,213.384155,23.418386,43.071693,32.324692,35.273739,45.940834,24.760000,46.691429,119.004936
2017-03-07,51.034359,84.277863,121.185951,87.172951,81.127777,143.103287,73.501015,41.446583,141.429993,58.648251,...,149.469559,212.444275,23.172947,43.368027,31.992146,35.229008,45.785866,24.480000,46.444466,119.124641
2017-03-08,51.537159,84.183197,121.932915,87.157021,80.901108,142.451981,73.970665,41.618958,140.320007,57.664673,...,149.679092,211.885468,23.118408,43.219856,32.104134,34.943817,45.452686,24.520000,46.372768,119.389221
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12-22,55.329586,107.189865,158.765381,96.554077,121.053329,167.901199,80.205994,52.816227,189.940002,62.300785,...,198.337997,221.460800,25.349760,42.606319,37.309025,38.582966,38.583553,25.100000,50.083778,150.370941
2017-12-26,55.321964,107.475220,160.647324,96.790192,121.943336,167.829224,80.085861,52.647831,187.759995,61.878906,...,197.977371,220.391830,25.398849,42.570881,37.113678,38.455517,38.203815,25.129999,49.742020,150.832184
2017-12-27,55.535053,108.454933,160.503845,97.123970,122.696671,167.604324,80.291847,52.280647,186.240005,62.160156,...,198.716629,218.878189,25.483032,42.777599,36.678619,38.808899,38.433243,25.200001,49.595554,151.312637


## 3. Mean, Volatility and Covariance

In [5]:
names, cov, annualized_returns = get_matrices(raw_data, max_combination, None)[0]
volatility = np.sqrt(np.diag(cov))

risk_free_rate=0 
sharpe_ratios = (annualized_returns - risk_free_rate) / volatility

In [6]:
hover_texts = [
    f"{ticker} <br>Volatility: {vol:.2f} <br>Returns: {ret:.2%} <br>Sharpe Ratio: {sr:.2f}"
    for ticker, vol, ret, sr in zip(names, volatility, annualized_returns, sharpe_ratios)
]

fig = go.Figure(data=go.Scatter(
    x=volatility, 
    y=annualized_returns, 
    mode='markers',
    hoverinfo='text',
    hovertext=hover_texts,
    marker=dict(color=sharpe_ratios, colorscale = 'RdBu', size=6, line=dict(width=1), colorbar=dict(title="Sharpe<br>Ratio")
    )
))

fig.update_layout(
    title='Markowitz Mean Varience Model',
    xaxis_title='Volatility (Standard Deviation)',
    yaxis_title='Annualized Returns',
)

fig.show()


In [7]:
all_portfolios, dominant_portfolios = MonteCarloRBA(names, cov, annualized_returns, 10000)

100%|██████████| 10000/10000 [00:06<00:00, 1510.45it/s]


In [8]:
len(dominant_portfolios)
len(all_portfolios)

10000

In [9]:
fig1 = go.Figure()

fig1.add_trace(go.Scatter(
    x=[p["variance"]**0.5 for p in all_portfolios],
    y=[p["return"] for p in all_portfolios],
    mode='markers',
    marker=dict(
        color=[p["return"] / (p["variance"]**0.5) for p in all_portfolios],
        showscale=True,
        size=7,
        line=dict(width=1),
        colorscale="RdBu",
        colorbar=dict(title="Sharpe<br>Ratio")
    ),
    hoverinfo='text',
    text=[
        f"Return: {p['return']:.2%}<br>Volatility: {p['variance']**0.5:.2f}<br>" +
        "<br>".join([f"{p['tickers'][i]}: Weight={p['weights'][i]:.2f}" for i in range(len(p['tickers']))])
        for p in all_portfolios
    ]
))

fig1.update_layout(
    xaxis=dict(title='Volatility (Standard Deviation)'),
    yaxis=dict(title='Annualised Returns'),
    title='Sample of Random Portfolios'
)

fig1.show()

In [10]:
fig2 = go.Figure()

fig2.add_trace(go.Scatter(
    x=[p["variance"]**0.5 for p in dominant_portfolios],  # Convert variance to volatility
    y=[p["return"] for p in dominant_portfolios],
    mode='markers',
    marker=dict(
        color=[p["return"] / (p["variance"]**0.5) for p in dominant_portfolios],  # Sharpe Ratio
        showscale=True,
        size=7,
        line=dict(width=1),
        colorscale="RdBu",
        colorbar=dict(title="Sharpe<br>Ratio")
    ),
    hoverinfo='text',
    text=[
        f"Return: {p['return']:.2%}<br>Volatility: {p['variance']**0.5:.2f}<br>" +
        "<br>".join([f"{p['tickers'][i]}: Weight={p['weights'][i]:.2f}" for i in range(len(p['tickers']))])
        for p in dominant_portfolios
    ],
    name="Portfolios"
))


fig2.add_trace(go.Scatter(
    x=volatility, 
    y=annualized_returns,
    mode='markers',
    hoverinfo='text',
    hovertext=[
        f"{name} <br>Volatility: {vol:.2f} <br>Returns: {ret:.2%} <br>Sharpe Ratio: {sr:.2f}"
        for name, vol, ret, sr in zip(names, volatility, annualized_returns, sharpe_ratios)
    ],
    marker=dict(
        color='brown',
        size=5,
        symbol='triangle-up',  # Sets the marker shape to a triangle
        line=dict(width=1)
    ),
    name="Individual Assets"
))

fig2.update_layout(
    title='Sample of Random Portfolios',
    xaxis_title='Volatility (Standard Deviation)',
    yaxis_title='Annualized Return',
    legend=dict(y=5
    )
)


fig2.show()